# MetPy Calculations

This notebook demonstrates a simple calculation for 1000-500-hPa thickness by adding it to the DataArray.

### Import Packages

In [ ]:
from datetime import datetime, timedelta

import metpy.calc as mpcalc
from metpy.plots import declarative
from metpy.units import units
import numpy as np
import xarray as xr

### Get Data

In [ ]:
# Set the date/time of the model run
date = datetime(2023, 3, 3, 12)

# Remote access to the dataset from the UCAR site
ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib'
                     f'/NCEP/GFS/Global_onedeg/GFS_Global_onedeg_'
                     f'{date:%Y%m%d}_{date:%H%M}.grib2')

# Subset data to be just over the U.S. for plotting purposes and for the plot_time
ds = ds.sel(lat=slice(70,10), lon=slice(360-140, 360-60))

# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

### A Simple Calculation

1000-500-hPa Thickness = (Heights at 500 hPa) - (Heights at 1000 hPa)

Easy to do subtraction with a data array and store it directly back into the dataset (ds). In order to do this well, we'll need to quantify the variable (e.g., attach units to the variable we're pulling from our Dataset. Then we'll do the calculation and dequantify it so we are back in a more native xarray DataArray format.
                   

In [ ]:
hght_500 = ds.Geopotential_height_isobaric.metpy.sel(time=plot_time,
                                                     vertical=500 * units.hPa).metpy.quantify()
hght_1000 = ds.Geopotential_height_isobaric.metpy.sel(time=plot_time,
                                                      vertical=1000 * units.hPa).metpy.quantify()
ds['thickness'] = (hght_500 - hght_1000).metpy.dequantify()

### Plot Data

Note that since we subset by time, as well as by vertical level, when isolating the 500 and 1000-hPa geopotential heights above, we no longer need to subst by those dimensions when plotting. So we'll need to set the *level* and *time* attributes to ```None```.

In [ ]:
# Set attributes for plotting contours
cntr = declarative.ContourPlot()
cntr.data = ds
cntr.field = 'thickness'
cntr.level = None
cntr.time = None
cntr.contours = list(range(0, 10000, 60))
cntr.linecolor = 'red'
cntr.linestyle = 'dashed'
cntr.clabels = True

cntr2 = declarative.ContourPlot()
cntr2.data = ds
cntr2.field = 'Geopotential_height_isobaric'
cntr2.level = 500 * units.hPa
cntr2.time = plot_time
cntr2.contours = range(0, 10000, 60)
cntr2.linecolor = 'black'
cntr2.linestyle = 'solid'
cntr2.clabels = True

# Set the attributes for the map
# and put the contours on the map
panel = declarative.MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'1000-500-hPa Thickness at {plot_time} by KHG'
panel.plots = [cntr, cntr2]

# Set the attributes for the panel
# and put the panel in the figure
pc = declarative.PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()

### Scaling Data

Since vorticity values are on the order of 10^-5 we want to scale them to fit common contouring values of whole numbers, so we want to set the `scale` attribute to be `1e5`.

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

# Set attributes for plotting contours
cfill = declarative.FilledContourPlot()
cfill.data = ds
cfill.field = 'Absolute_vorticity_isobaric'
cfill.level = 500 * units.hPa
cfill.time = plot_time
cfill.contours = list(range(-46, 47, 2))
cfill.colormap = 'PuOr_r'
cfill.colorbar = 'horizontal'
cfill.scale = 1e5
cfill.smooth_field = 3

cntr2 = declarative.ContourPlot()
cntr2.data = ds
cntr2.field = 'Geopotential_height_isobaric'
cntr2.level = 500 * units.hPa
cntr2.time = plot_time
cntr2.contours = list(range(0, 10000, 60))
cntr2.linecolor = 'black'
cntr2.linestyle = 'solid'
cntr2.clabels = True

# Set the attributes for the map
# and put the contours on the map
panel = declarative.MapPanel()
panel.area = [-125, -74, 22, 52]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'{cfill.level.m}-hPa Abs. Vorticity at {plot_time} by KHG'
panel.plots = [cfill, cntr2]

# Set the attributes for the panel
# and put the panel in the figure
pc = declarative.PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()